In [1]:
import h5py
import pandas

In [2]:
import sparkhpc
from sparkhpc import sparkjob
import findspark 
findspark.init("/cluster/home/simondi/spark/")
import pyspark

In [68]:
from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType

In [16]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [3]:
sc = pyspark.SparkContext("local", "App Name")

In [4]:
file_name = "/cluster/home/simondi/simondi/tix/data/screening_data/cells_sample_10.tsv"

In [14]:
data = sc.textFile(file_name)
data.take(5)

['study\tpathogen\tlibrary\tdesign\treplicate\tplate\twell\tgene\tsirna\twell_type\timage_idx\tobject_idx\tcells.areashape_area\tcells.areashape_eccentricity\tcells.areashape_extent\tcells.areashape_formfactor\tcells.areashape_majoraxislength\tcells.areashape_minoraxislength\tcells.areashape_perimeter\tcells.children_bacteria_count\tcells.children_invasomes_count\tcells.location_center_x\tcells.location_center_y\tcells.neighbors_anglebetweenneighbors_2\tcells.neighbors_firstclosestobjectnumber_2\tcells.neighbors_firstclosestxvector_2\tcells.neighbors_firstclosestyvector_2\tcells.neighbors_numberofneighbors_2\tcells.neighbors_secondclosestobjectnumber_2\tcells.neighbors_secondclosestxvector_2\tcells.neighbors_secondclosestyvector_2\tcells.parent_nuclei',
 'infectx_published\tlisteria\ta\tu\t1\tkb2-02-1x\ta01\tchka\ts3008\tsirna\t4\t144\t2842.0\t0.751864254475\t0.656047999859\t0.61208987236\t75.3157806396\t49.6570472717\t240.55130004900002\t0.0\t0\t881.00177002\t910.962341309\t171.973251

In [24]:
#data_rdd = data.map(lambda line: line.split("\t"))
df = sqlc.read.format('com.databricks.spark.csv').options(header='true',delimiter='\t', inferschema='true').load(file_name)

In [85]:
old_cols = df.schema.names
new_cols = list(map(lambda x: x.replace(".", "_"),  old_cols))
df = reduce(lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]), range(len(new_cols)), df)
for i, x in enumerate(new_cols):
    if x.startswith("cells"):
        df= df.withColumn(x, df[x].cast("double"))
df.printSchema()

root
 |-- study: string (nullable = true)
 |-- pathogen: string (nullable = true)
 |-- library: string (nullable = true)
 |-- design: string (nullable = true)
 |-- replicate: integer (nullable = true)
 |-- plate: string (nullable = true)
 |-- well: string (nullable = true)
 |-- gene: string (nullable = true)
 |-- sirna: string (nullable = true)
 |-- well_type: string (nullable = true)
 |-- image_idx: integer (nullable = true)
 |-- object_idx: integer (nullable = true)
 |-- cells_areashape_area: double (nullable = true)
 |-- cells_areashape_eccentricity: double (nullable = true)
 |-- cells_areashape_extent: double (nullable = true)
 |-- cells_areashape_formfactor: double (nullable = true)
 |-- cells_areashape_majoraxislength: double (nullable = true)
 |-- cells_areashape_minoraxislength: double (nullable = true)
 |-- cells_areashape_perimeter: double (nullable = true)
 |-- cells_children_bacteria_count: double (nullable = true)
 |-- cells_children_invasomes_count: double (nullable = tru

In [86]:
s = df.select("cells_parent_nuclei")
s.take(1)

[Row(cells_parent_nuclei=144.0)]

In [90]:
df.take(4)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate=1, plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx=4, object_idx=144, cells_areashape_area=2842.0, cells_areashape_eccentricity=0.751864254475, cells_areashape_extent=0.656047999859, cells_areashape_formfactor=0.61208987236, cells_areashape_majoraxislength=75.3157806396, cells_areashape_minoraxislength=49.6570472717, cells_areashape_perimeter=240.55130004900002, cells_children_bacteria_count=0.0, cells_children_invasomes_count=0.0, cells_location_center_x=881.00177002, cells_location_center_y=910.962341309, cells_neighbors_anglebetweenneighbors_2=171.973251343, cells_neighbors_firstclosestobjectnumber_2=145.0, cells_neighbors_firstclosestxvector_2=30.1583766937, cells_neighbors_firstclosestyvector_2=-53.266174316400004, cells_neighbors_numberofneighbors_2=5.0, cells_neighbors_secondclosestobjectnumber_2=134.0, cells_neighbors_secondclosestxvector_2=-25.2489166

In [89]:
vectorial= df.rdd.map(lambda line: line[12:])
vectorial.take(4)

[(2842.0,
  0.751864254475,
  0.656047999859,
  0.61208987236,
  75.3157806396,
  49.6570472717,
  240.55130004900002,
  0.0,
  0.0,
  881.00177002,
  910.962341309,
  171.973251343,
  145.0,
  30.1583766937,
  -53.266174316400004,
  5.0,
  134.0,
  -25.248916626,
  64.1275634766,
  144.0),
 (2484.0,
  0.7443224787710001,
  0.564288973808,
  0.48369204998000004,
  73.0543823242,
  48.7872047424,
  253.036575317,
  1.0,
  0.0,
  872.33215332,
  455.98107910199997,
  97.5600357056,
  165.0,
  -17.180620193499998,
  42.949043273899996,
  4.0,
  155.0,
  -54.698463439899996,
  -30.773979187,
  168.0),
 (4646.0,
  0.91317820549,
  0.715870559216,
  0.518087804317,
  122.713951111,
  50.013359069799996,
  334.69342041,
  17.0,
  0.0,
  1025.27355957,
  179.84201049799998,
  143.13684082,
  212.0,
  -8.14504814148,
  45.5921897888,
  8.0,
  223.0,
  36.486579895,
  -34.0335769653,
  210.0),
 (4070.0,
  0.852616667747,
  0.544117629528,
  0.517207980156,
  101.693138123,
  53.138420105,
  313.

In [71]:
s.take(1)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate=1, plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx=4, object_idx=144, cells_areashape_area=2842.0, cells_areashape_eccentricity='0.751864254475', cells_areashape_extent='0.656047999859', cells_areashape_formfactor='0.61208987236', cells_areashape_majoraxislength='75.3157806396', cells_areashape_minoraxislength='49.6570472717', cells_areashape_perimeter='240.55130004900002', cells_children_bacteria_count=0.0, cells_children_invasomes_count='0', cells_location_center_x='881.00177002', cells_location_center_y='910.962341309', cells_neighbors_anglebetweenneighbors_2='171.973251343', cells_neighbors_firstclosestobjectnumber_2='145.0', cells_neighbors_firstclosestxvector_2='30.1583766937', cells_neighbors_firstclosestyvector_2='-53.266174316400004', cells_neighbors_numberofneighbors_2='5.0', cells_neighbors_secondclosestobjectnumber_2='134.0', cells_neighbors_second